In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
from Mapping import *
from statsmodels.formula.api import ols
import seaborn as sns

In [ ]:
data = pd.read_csv("/content/gdrive/MyDrive/ParticipantData12Dec2022.csv")

In [ ]:
data.head()

,participant_id,video_id,recording_elegible,cam_allowed,azure_features,video_error,duration_recording,survey_duration,pre_quiz1,pre_quiz2,...,correct_post_quiz,self_raw_anger,self_raw_contempt,self_raw_disgust,self_raw_fear,self_raw_happiness,self_raw_neutral,self_raw_sadness,self_raw_surprise,reported_emotions
0,5391298,13201,0,0.0,0.0,0,NaN,767,a,d,...,4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,5202198,13201,0,0.0,0.0,0,NaN,536,a,c,...,8,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,415738,13201,0,0.0,0.0,0,NaN,665,a,c,...,4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,8137098,13201,0,0.0,0.0,0,NaN,1137,d,c,...,4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,3651718,13201,0,0.0,0.0,0,NaN,318,a,c,...,6,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [ ]:
data["compliers"] = np.where((data['recording_elegible']==1) & (data['cam_allowed'] == 1),1,0)

In [ ]:
compliers = len(data[data["compliers"]==1])

In [ ]:
compliers

98

##98 compliers: They were asked to keep their videos on and camera was allowed.

In [ ]:
data[['recording_elegible','cam_allowed','azure_features','video_error']].value_counts()

recording_elegible  cam_allowed  azure_features  video_error
0                   0.0          0.0             0              141
1                   1.0          1.0             0               61
                                 0.0             1               25
                    0.0          0.0             0               15
                    1.0          0.0             0               12
dtype: int64

- Control group - never offered to record: 141
- Compliers Total: 98
   1. Perfect Video (atleast 1 frame): 61
   2. No face shown: 12
   3. File corrupted: 25
- Non Compliers: 15


In [ ]:
data[['azure_features','video_error']].value_counts()

azure_features  video_error
0.0             0              168
1.0             0               61
0.0             1               25
dtype: int64

## Complier Split up
- For 168 survey takers, the recording was required and camera was allowed but no face was detected.
- For 25 survey takers, the recording was corrupted after they agreed to record.
- For 61 survey takers, the recording was perfect.


In [ ]:
data.groupby("compliers")["correct_pre_quiz"].mean()

compliers
0    1.250000
1    1.204082
Name: correct_pre_quiz, dtype: float64

## Average score of compliers before watching the video was pretty low around 1.25 for both the compliers and non-compliers(control group).

In [ ]:
data.groupby("compliers")["correct_post_quiz"].mean()

compliers
0    4.743590
1    4.857143
Name: correct_post_quiz, dtype: float64

## Average score of compliers after watching the video -

1. Recording themselves: 4.85
2. Without recording: 4.74

In [ ]:
data.groupby(["recording_elegible","cam_allowed","azure_features","video_error"])["correct_pre_quiz"].mean()

recording_elegible  cam_allowed  azure_features  video_error
0                   0.0          0.0             0              1.255319
1                   0.0          0.0             0              1.200000
                    1.0          0.0             0              0.583333
                                                 1              1.160000
                                 1.0             0              1.344262
Name: correct_pre_quiz, dtype: float64

In [ ]:
data.groupby(["recording_elegible","cam_allowed","azure_features","video_error"])["correct_post_quiz"].mean()

recording_elegible  cam_allowed  azure_features  video_error
0                   0.0          0.0             0              4.765957
1                   0.0          0.0             0              4.533333
                    1.0          0.0             0              4.416667
                                                 1              4.640000
                                 1.0             0              5.032787
Name: correct_post_quiz, dtype: float64

## Gender wise averages

In [ ]:
data.groupby(["demo_gender","recording_elegible","cam_allowed","azure_features","video_error"])["correct_pre_quiz"].mean().reset_index()

,demo_gender,recording_elegible,cam_allowed,azure_features,video_error,correct_pre_quiz
0,a,0,0.0,0.0,0,1.270270
1,a,1,0.0,0.0,0,1.375000
2,a,1,1.0,0.0,0,0.714286
3,a,1,1.0,0.0,1,1.000000
4,a,1,1.0,1.0,0,1.393939
5,b,0,0.0,0.0,0,1.333333
6,b,1,0.0,0.0,0,1.200000
7,b,1,1.0,0.0,0,0.500000
8,b,1,1.0,0.0,1,1.363636
9,b,1,1.0,1.0,0,1.360000


## Male, Female and Other gender wise average pre video scores above

In [ ]:
data.groupby(["demo_gender","recording_elegible","cam_allowed","azure_features","video_error"])["correct_post_quiz"].mean().reset_index()

,demo_gender,recording_elegible,cam_allowed,azure_features,video_error,correct_post_quiz
0,a,0,0.0,0.0,0,5.067568
1,a,1,0.0,0.0,0,5.125000
2,a,1,1.0,0.0,0,4.428571
3,a,1,1.0,0.0,1,5.230769
4,a,1,1.0,1.0,0,4.727273
5,b,0,0.0,0.0,0,4.789474
6,b,1,0.0,0.0,0,3.800000
7,b,1,1.0,0.0,0,4.250000
8,b,1,1.0,0.0,1,4.363636
9,b,1,1.0,1.0,0,5.520000


## Male, Female and Other gender wise average post video scores above

In [ ]:
data = pd.read_csv("/content/gdrive/MyDrive/ParticipantData12Dec2022.csv")

In [ ]:
data["compliers"] = np.where((data['recording_elegible']==1) & (data['cam_allowed'] == 1),1,0)

In [ ]:
data["correct_pre_quiz"] = data["correct_pre_quiz"]/2
data["correct_post_quiz"] = data["correct_post_quiz"]/2

# ITT and LATE

In [ ]:
data.columns

Index(['participant_id', 'video_id', 'recording_elegible', 'cam_allowed',
       'azure_features', 'video_error', 'duration_recording',
       'survey_duration', 'pre_quiz1', 'pre_quiz2', 'post_quiz1', 'post_quiz2',
       'post_quiz3', 'post_quiz4', 'post_quiz5', 'post_quiz6', 'post_quiz7',
       'post_quiz8', 'self_interest', 'self_boring', 'self_confusion',
       'self_easy', 'demo_age', 'demo_gender', 'demo_race', 'demo_ethnicity',
       'demo_marital_status', 'demo_education', 'demo_income',
       'demo_employment', 'demo_industry', 'demo_country', 'demo_region',
       'correct', 'correct_pre_quiz', 'correct_post_quiz', 'self_raw_anger',
       'self_raw_contempt', 'self_raw_disgust', 'self_raw_fear',
       'self_raw_happiness', 'self_raw_neutral', 'self_raw_sadness',
       'self_raw_surprise', 'reported_emotions', 'compliers'],
      dtype='object')

In [ ]:
data = data.replace({"demo_gender":demo_gender,"demo_age":demo_age,"demo_marital_status":demo_marital_status,"demo_education":demo_education,
                     "demo_income":demo_income,"demo_employment":demo_employment})

In [ ]:
treated_effect_0 = np.mean(data[(data["recording_elegible"]==0)]["correct_post_quiz"])
treated_effect_1 = np.mean(data[(data["recording_elegible"]==1)]["correct_post_quiz"])

In [ ]:
treated_effect_0

2.382978723404255

In [ ]:
treated_effect_1

2.4070796460176993

In [ ]:
ITT = treated_effect_1 - treated_effect_0

In [ ]:
ITT

0.024100922613444098

### The average effect of treatment on score is 4.85 and the average effect of control on score is 4.74. So, the ITT can be defined as:

### ITT = [avg(Y)|T=1] - [avg(Y)|T=0] = +0.048

In [ ]:
LATE = ITT/(len(data[(data["compliers"]==1)])/len(data[(data["recording_elegible"]==1)]))

In [ ]:
LATE

0.027789839339991664

LATE in this scenario is ITT/(% of Compliers in the dataset)

LATE is approximately +0.027

In [ ]:
# % of never takers
p_never_takers = len(data[(data["recording_elegible"]==1) & (data["cam_allowed"]==0)])/len(data[(data["recording_elegible"]==1)])
p_never_takers

0.13274336283185842

In [ ]:
# % of compliers
p_compliers = (len(data[(data["compliers"]==1)])/len(data[(data["recording_elegible"]==1)]))
p_compliers

0.8672566371681416

In [ ]:
# Effect on Never Takers
y_never_takers = np.mean(data[(data["recording_elegible"]==1) & (data["cam_allowed"]==0)]["correct_post_quiz"])

In [ ]:
y_never_takers

2.2666666666666666

In [ ]:
# Effect on treated compliers
y_treated_compliers = np.mean(data[(data["recording_elegible"]==1) & (data["cam_allowed"]==1)]["correct_post_quiz"])

In [ ]:
y_treated_compliers

2.4285714285714284

In [ ]:
# Effect on control compliers
y_control_compliers = (treated_effect_0-p_never_takers*y_never_takers)/p_compliers
y_control_compliers

2.4007815892314373

In [ ]:
y_treated_compliers - y_control_compliers

0.02778983933999113

# OLS for ITT

### Without Heterogeneous Effect

In [ ]:
# fit simple linear regression model
# linear_model = ols('correct_post_quiz ~ recording_elegible+cam_allowed+azure_features+video_error+survey_duration+C(demo_gender)+C(demo_age)'+
#                    '+C(demo_marital_status)+C(demo_education)+C(demo_income)+C(demo_employment)+correct_pre_quiz',
#                    data=data).fit()
# linear_model = ols('correct_post_quiz ~ recording_elegible+cam_allowed+azure_features+video_error+survey_duration+correct_pre_quiz+C(demo_gender)',
#                    data=data).fit()
linear_model = ols('correct_post_quiz ~ recording_elegible',
                   data=data).fit()
# display model summary
print(linear_model.summary())

                            OLS Regression Results                            
Dep. Variable:      correct_post_quiz   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                   0.04200
Date:                Sat, 17 Dec 2022   Prob (F-statistic):              0.838
Time:                        03:10:05   Log-Likelihood:                -341.35
No. Observations:                 254   AIC:                             686.7
Df Residuals:                     252   BIC:                             693.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              2.3830      0

### With Heterogeneous Effect

In [ ]:
data = pd.get_dummies(data, prefix=['demo_gender'], columns=['demo_gender'])
data.columns
data.columns = data.columns.str.replace(" ","_")
data.columns

Index(['participant_id', 'video_id', 'recording_elegible', 'cam_allowed',
       'azure_features', 'video_error', 'duration_recording',
       'survey_duration', 'pre_quiz1', 'pre_quiz2', 'post_quiz1', 'post_quiz2',
       'post_quiz3', 'post_quiz4', 'post_quiz5', 'post_quiz6', 'post_quiz7',
       'post_quiz8', 'self_interest', 'self_boring', 'self_confusion',
       'self_easy', 'demo_age', 'demo_race', 'demo_ethnicity',
       'demo_marital_status', 'demo_education', 'demo_income',
       'demo_employment', 'demo_industry', 'demo_country', 'demo_region',
       'correct', 'correct_pre_quiz', 'correct_post_quiz', 'self_raw_anger',
       'self_raw_contempt', 'self_raw_disgust', 'self_raw_fear',
       'self_raw_happiness', 'self_raw_neutral', 'self_raw_sadness',
       'self_raw_surprise', 'reported_emotions', 'compliers',
       'demo_gender_Female', 'demo_gender_Male', 'demo_gender_Other',
       'demo_gender_Prefer_not_to_say'],
      dtype='object')

In [ ]:
data["Male_Female"] = data["demo_gender_Male"]+(-1*data["demo_gender_Female"])
data["Male_Female"].replace(-1,0,inplace=True)
data.head()

,participant_id,video_id,recording_elegible,cam_allowed,azure_features,video_error,duration_recording,survey_duration,pre_quiz1,pre_quiz2,...,self_raw_neutral,self_raw_sadness,self_raw_surprise,reported_emotions,compliers,demo_gender_Female,demo_gender_Male,demo_gender_Other,demo_gender_Prefer_not_to_say,Male_Female
0,5391298,13201,0,0.0,0.0,0,NaN,767,a,d,...,1.0,0.0,0.0,1.0,0,0,1,0,0,1
1,5202198,13201,0,0.0,0.0,0,NaN,536,a,c,...,1.0,0.0,0.0,1.0,0,1,0,0,0,0
2,415738,13201,0,0.0,0.0,0,NaN,665,a,c,...,1.0,0.0,0.0,1.0,0,0,1,0,0,1
3,8137098,13201,0,0.0,0.0,0,NaN,1137,d,c,...,0.0,0.0,0.0,1.0,0,0,1,0,0,1
4,3651718,13201,0,0.0,0.0,0,NaN,318,a,c,...,1.0,0.0,0.0,1.0,0,0,1,0,0,1


In [ ]:

linear_model_hg = ols('correct_post_quiz ~ recording_elegible+Male_Female+recording_elegible*Male_Female',
                   data=data).fit()
print(linear_model_hg.summary())

                            OLS Regression Results                            
Dep. Variable:      correct_post_quiz   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1.406
Date:                Sat, 17 Dec 2022   Prob (F-statistic):              0.242
Time:                        03:10:07   Log-Likelihood:                -339.24
No. Observations:                 254   AIC:                             686.5
Df Residuals:                     250   BIC:                             700.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept   

## LATE with IVREG

In [ ]:

# data["Male/Female"] = data["demo_gender_Male"]-data["demo_gender_Female"]
# data["Male/Female"].replace(-1,0)


In [ ]:
# data["Male/Female"] = np.where((data['demo_gender']=="Male") | ((data['demo_gender'] !="Prefer not to say")&(data['demo_gender'] !="Other")),1,0)
# data = pd.get_dummies(data[(data["recording_elegible"]==1)], prefix=['demo_gender'], columns=['demo_gender'])
# data = pd.get_dummies(data[(data["recording_elegible"]==1)], prefix=['demo_gender'], columns=['demo_gender'])
# data.columns
# data.columns = data.columns.str.replace(" ","_")
# data.columns
# data=data[(data["recording_elegible"]==1)]
data.columns

Index(['participant_id', 'video_id', 'recording_elegible', 'cam_allowed',
       'azure_features', 'video_error', 'duration_recording',
       'survey_duration', 'pre_quiz1', 'pre_quiz2', 'post_quiz1', 'post_quiz2',
       'post_quiz3', 'post_quiz4', 'post_quiz5', 'post_quiz6', 'post_quiz7',
       'post_quiz8', 'self_interest', 'self_boring', 'self_confusion',
       'self_easy', 'demo_age', 'demo_race', 'demo_ethnicity',
       'demo_marital_status', 'demo_education', 'demo_income',
       'demo_employment', 'demo_industry', 'demo_country', 'demo_region',
       'correct', 'correct_pre_quiz', 'correct_post_quiz', 'self_raw_anger',
       'self_raw_contempt', 'self_raw_disgust', 'self_raw_fear',
       'self_raw_happiness', 'self_raw_neutral', 'self_raw_sadness',
       'self_raw_surprise', 'reported_emotions', 'compliers',
       'demo_gender_Female', 'demo_gender_Male', 'demo_gender_Other',
       'demo_gender_Prefer_not_to_say', 'Male_Female'],
      dtype='object')

In [ ]:

# # late_data = data[(data["recording_elegible"]==1)&(data["cam_allowed"]==1)]

# lm1 = ols('recording_elegible ~ Male_Female',
#                    data=data).fit()

# print(lm1.summary())

                            OLS Regression Results                            
Dep. Variable:     recording_elegible   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                   0.05625
Date:                Sat, 17 Dec 2022   Prob (F-statistic):              0.813
Time:                        03:10:22   Log-Likelihood:                -182.77
No. Observations:                 254   AIC:                             369.5
Df Residuals:                     252   BIC:                             376.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.4370      0.046      9.555      

In [ ]:
# lm2 = ols('demo_gender_Female ~ cam_allowed',
#                    data=data).fit()

# print(lm2.summary())

                            OLS Regression Results                            
Dep. Variable:     demo_gender_Female   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                   0.02860
Date:                Sat, 17 Dec 2022   Prob (F-statistic):              0.866
Time:                        03:10:31   Log-Likelihood:                -179.32
No. Observations:                 254   AIC:                             362.6
Df Residuals:                     252   BIC:                             369.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.3974      0.039     10.087      

In [ ]:
# # print(late_data.columns)
# # # late_data.rename(columns={"demo_gender_Prefer not to say":"demo_gender_Prefer_not_to_say"},inplace=True)
# lm3 = ols('demo_gender_Prefer_not_to_say ~ recording_elegible',
#                    data=data).fit()

# print(lm3.summary())

                                  OLS Regression Results                                 
Dep. Variable:     demo_gender_Prefer_not_to_say   R-squared:                       0.000
Model:                                       OLS   Adj. R-squared:                  0.000
Method:                            Least Squares   F-statistic:                       nan
Date:                           Sat, 17 Dec 2022   Prob (F-statistic):                nan
Time:                                   01:45:57   Log-Likelihood:                 30.468
No. Observations:                            113   AIC:                            -58.94
Df Residuals:                                112   BIC:                            -56.21
Df Model:                                      0                                         
Covariance Type:                       nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
-------------

In [ ]:
# lm4 = ols('demo_gender_Other ~ recording_elegible',
#                    data=data).fit()

# print(lm4.summary())

                            OLS Regression Results                            
Dep. Variable:      demo_gender_Other   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Sat, 17 Dec 2022   Prob (F-statistic):                nan
Time:                        01:45:58   Log-Likelihood:                 46.206
No. Observations:                 113   AIC:                            -90.41
Df Residuals:                     112   BIC:                            -87.68
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              0.0133      0

### First and Second Stage

In [ ]:
import statsmodels.api as sm
from linearmodels.iv import IV2SLS

# from __future__ import division
# # data = data[(data["recording_elegible"]==1)]
# # endog = data[["correct_post_quiz"]]
# # exog = data[["demo_gender_Male","demo_gender_Female","demo_gender_Other","demo_gender_Prefer_not_to_say"]]
# # instr = data[["recording_elegible","cam_allowed","survey_duration","correct_pre_quiz","azure_features","video_error"]]
# res_first = IV2SLS(data[["recording_elegible"]], data[["demo_gender_Male","demo_gender_Female","demo_gender_Other","demo_gender_Prefer_not_to_say"]], None, None).fit(
#     cov_type="unadjusted")
# print(res_first)
# # print(no_endog_results.summary())
# data["cam_hat"] = data.cam_allowed - res_first.resids

In [ ]:
# from statsmodels.api import add_constant
# data = add_constant(data, has_constant="add")
# res_second = IV2SLS(data[["correct_post_quiz"]], data[["survey_duration","correct_pre_quiz","azure_features","video_error","recording_elegible"]],
#                     data[["cam_allowed"]], data[["demo_gender_Male","demo_gender_Female","demo_gender_Other","demo_gender_Prefer_not_to_say"]]).fit(
#     cov_type="unadjusted"
# )
# print(res_second)

In [ ]:
# # dep ~ exog + [ endog ~ instruments]
# formula = (
#     "ldrugexp ~ 1 + totchr + female + age + linc + blhisp + [hi_empunion ~ ssiratio]"
# )
# mod = IV2SLS.from_formula(formula, data)
formula = ("correct_post_quiz ~ 1+recording_elegible+survey_duration+correct_pre_quiz+azure_features+video_error+[cam_allowed ~ Male_Female]")
mod = IV2SLS.from_formula(formula,data)
iv_res = mod.fit(cov_type="robust")
print(iv_res)
# print(mod)

                          IV-2SLS Estimation Summary                          
Dep. Variable:      correct_post_quiz   R-squared:                     -24.664
Estimator:                    IV-2SLS   Adj. R-squared:                -25.288
No. Observations:                 254   F-statistic:                    3.3721
Date:                Sat, Dec 17 2022   P-value (F-stat)                0.7609
Time:                        03:22:42   Distribution:                  chi2(6)
Cov. Estimator:                robust                                         
                                                                              
                                 Parameter Estimates                                  
                    Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
--------------------------------------------------------------------------------------
Intercept              0.5552     3.6717     0.1512     0.8798     -6.6412      7.7517
azure_features      

In [ ]:
res_ols = IV2SLS(data.correct_post_quiz, data[["cam_allowed","survey_duration","correct_pre_quiz","azure_features","video_error",
                                               "demo_gender_Male","demo_gender_Female","demo_gender_Other","demo_gender_Prefer_not_to_say"]], None, None).fit(
    cov_type="unadjusted"
)
print(res_ols)

                            OLS Estimation Summary                            
Dep. Variable:      correct_post_quiz   R-squared:                      0.2272
Estimator:                        OLS   Adj. R-squared:                 0.1677
No. Observations:                 113   F-statistic:                    913.34
Date:                Sat, Dec 17 2022   P-value (F-stat)                0.0000
Time:                        01:48:11   Distribution:                  chi2(9)
Cov. Estimator:            unadjusted                                         
                                                                              
                                       Parameter Estimates                                       
                               Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------------------------
cam_allowed                       0.3351     0.3439     0.9744     0.3299 

In [ ]:
from linearmodels.iv import compare

res_direct = IV2SLS(data.correct_post_quiz, data[["cam_hat"]], None, None).fit(
    cov_type="unadjusted"
)
print(compare({"OLS": res_ols, "LATE_SecondStage_IV2SLS": res_second}))
# print(compare({"OLS": res_ols, "2SLS": res_second}))

                                Model Comparison                               
                                                    OLS LATE_SecondStage_IV2SLS
-------------------------------------------------------------------------------
Dep. Variable                         correct_post_quiz       correct_post_quiz
Estimator                                           OLS                 IV-2SLS
No. Observations                                    113                     113
Cov. Est.                                    unadjusted              unadjusted
R-squared                                        0.2272                  0.8108
Adj. R-squared                                   0.1677                  0.8021
F-statistic                                      913.34                  517.18
P-value (F-stat)                                 0.0000                  0.0000
===============================     ===================     ===================
cam_allowed                             

In [ ]:
df = pd.read_csv("/content/gdrive/MyDrive/ParticipantData12Dec2022.csv")

In [ ]:
df["compliers"] = np.where((df['recording_elegible']==1) & (df['cam_allowed'] == 1),1,0)

In [ ]:
df["correct_pre_quiz"] = df["correct_pre_quiz"]/2
df["correct_post_quiz"] = df["correct_post_quiz"]/2

In [ ]:
df = df.replace({"demo_gender":demo_gender,"demo_age":demo_age,"demo_marital_status":demo_marital_status,"demo_education":demo_education,
                     "demo_income":demo_income,"demo_employment":demo_employment})

In [ ]:
# fit simple linear regression model
# linear_model = ols('correct_post_quiz ~ recording_elegible+cam_allowed+azure_features+video_error+survey_duration+C(demo_gender)+C(demo_age)'+
#                    '+C(demo_marital_status)+C(demo_education)+C(demo_income)+C(demo_employment)+correct_pre_quiz',
#                    data=data).fit()
# linear_model = ols('correct_post_quiz ~ recording_elegible+cam_allowed+azure_features+video_error+survey_duration+correct_pre_quiz+C(demo_gender)',
#                    data=data).fit()
linear_model = ols('correct_post_quiz ~ recording_elegible',
                   data=df).fit()
# display model summary
print(linear_model.summary())

                            OLS Regression Results                            
Dep. Variable:      correct_post_quiz   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                   0.04200
Date:                Sat, 17 Dec 2022   Prob (F-statistic):              0.838
Time:                        01:24:37   Log-Likelihood:                -341.35
No. Observations:                 254   AIC:                             686.7
Df Residuals:                     252   BIC:                             693.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              2.3830      0

In [1]:
print(compare({"OLS_for_ITT":linear_model,"OLS_for_ITT_with_Heterogenity":res_ols,"LATE_FirstStage_IV2SLS": res_first, "LATE_SecondStage_IV2SLS": res_second}))

NameError: ignored